In [45]:
import logging
import pandas as pd
import numpy as np

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
url='https://en.wikipedia.org/wiki/Geologic_time_scale'

In [48]:
tables=pd.read_html(url)

In [49]:
print(len(tables))

23


In [50]:
tables[3].head()

,Name,Time Span,Etymology of name
0,Quaternary,2.6 to 0 million years ago,First introduced by Jules Desnoyers in 1829 fo...
1,Neogene,23 to 2.6 million years ago,Derived from the Greek words νέος (néos) meani...
2,Paleogene,66 to 23 million years ago,Derived from the Greek words παλιός (palaiós) ...
3,Cretaceous,145 to 66 million years ago,Derived from Terrain Crétacé used in 1822 by J...
4,Jurassic,201.3 to 145 million years ago,Named after the Jura Mountains. Originally use...


In [67]:
logging.basicConfig(format='%(asctime)s %(levelname)s %(name)s %(message)s', level=logging.INFO)

In [83]:
import downloader

In [84]:
import download_datasets

In [85]:
dfs=download_datasets.get_datasets()

----------------------------------------------------------------------------------
Processing data_sources/EuropeanMeanTemperatureSinceRomanTime_EuroMed2k.toml
   Year      Mean  Lower2sigma  Upper2sigma
0  -138 -0.037309     -0.85472      0.76376
1  -137 -0.029138     -0.92172      0.81199
2  -136 -0.066622     -0.91555      0.74954
3  -135 -0.435790     -1.27780      0.35291
4  -134 -0.503160     -1.32540      0.27134
...
      Year     Mean  Lower2sigma  Upper2sigma
2136  1999  0.78958      0.13234       1.5786
2137  2000  0.85275      0.21043       1.6025
2138  2001  0.94781      0.27156       1.8178
2139  2002  1.38810      0.73063       2.0861
2140  2003  2.26200      1.54560       2.9375
>>> euromed2k
----------------------------------------------------------------------------------
Processing data_sources/DWDGermanJulyMeanTemperature.toml
   Jahr  Monat  Brandenburg/Berlin  Brandenburg  Baden-Wuerttemberg  Bayern  \
0  1881      7               19.19        19.16               

In [76]:
dfs.keys()

dict_keys([])

In [ ]:
dfs['sunspots11000yrs']

In [ ]:
em2k=dfs['euromed2k'].to_numpy()
gjmt6=dfs['germany_june_meantemp'].to_numpy()
gjmt7=dfs['germany_july_meantemp'].to_numpy()
gjmt8=dfs['germany_august_meantemp'].to_numpy()
sunsp=dfs['sunspots11000yrs'].to_numpy()

In [ ]:
sunsp[:,0]=1950-sunsp[:,0]

In [ ]:
sunsp[:,1]

In [ ]:
gjmt=(gjmt6[:-1,:]+gjmt7+gjmt8)/3.0

In [ ]:
np.mean(gjmt[:,2])

In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
gjmean=np.mean(gjmt[:,2])

In [ ]:
win=100
em2kmat=moving_average(em2k[:,1],2*win+1)
gjmtmat=moving_average(gjmt[:,2]-gjmean,2*win+1)

In [ ]:
sunma=moving_average(sunsp[:,1],2*win+1)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14,8]  # inch! :-/

In [ ]:
plt.plot(gjmt[:,0],gjmt[:,2]-gjmean)
plt.plot(dfnp[:,0],dfnp[:,1])
plt.plot(sunsp[-200:,0],sunsp[-200:,1]/60-0.4)

In [ ]:
# plt.plot(dfnp[:,0],dfnp[:,1])
plt.plot(gjmt[win:-win,0],gjmtmat)
plt.plot(dfnp[win:-win,0],em2kmat)
plt.plot(sunsp[-200:,0],sunma[-200:]/50-0.5)

In [ ]:
plt.plot(dfnp[-120+win:-win,0],em2kmat[-120+win:-win])
plt.plot(gjmt[win:-win,0],gjmtmat)

In [ ]:
a=-1500
b=-1400
plt.plot(dfnp[a:b,0],dfnp[a:b,1])
plt.plot(dfnp[a:b,0],gjmtmat[a-win:b-win])


In [ ]:
dl.get('https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwinpuzSjP74AhWCOuwKHeiOCrsQFnoECDIQAQ&url=ftp%3A%2F%2Fftp.ncdc.noaa.gov%2Fpub%2Fdata%2Fpaleo%2Fclimate_forcing%2Fsolar_variability%2Fsolanki2004-ssn.txt&usg=AOvVaw1tTDiUMZ93LQAYE4N8jt6A')